In [2]:
import cv2
import numpy as np
import gradio as gr

def cartoonize_image_logic(img, intensity):
    max_size = 1000
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        img = cv2.resize(img, (int(w * scale), int(h * scale)))
        
    if img is None:
        return None
    
    filter_iterations = int(intensity / 10) + 1
    sigmaColor = int(intensity * 2) + 10
    sigmaSpace = 7

    base_block = int(intensity / 5)
    blockSize = base_block if base_block % 2 == 1 else base_block + 1
    if blockSize < 3: blockSize = 3
    
    C = 2 

    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_small = cv2.pyrDown(cv2.pyrDown(img_bgr))

    for _ in range(filter_iterations):
        img_small = cv2.bilateralFilter(img_small, d=9, sigmaColor=sigmaColor, sigmaSpace=sigmaSpace)

    img_cartoon = cv2.pyrUp(cv2.pyrUp(img_small))
    img_cartoon = cv2.resize(img_cartoon, (img_bgr.shape[1], img_bgr.shape[0]))

    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.medianBlur(img_gray, 7)
    
    img_edge = cv2.adaptiveThreshold(
        img_blur, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        blockSize=blockSize,
        C=C
    )
    
    img_edge_colored = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2BGR)
    cartoon_bgr = cv2.bitwise_and(img_cartoon, img_edge_colored)
    
    return cv2.cvtColor(cartoon_bgr, cv2.COLOR_BGR2RGB)

with gr.Blocks(title="影像卡通化轉換器") as demo:
    gr.Markdown("# 🎨 影像卡通化轉換器")
    
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(label="1. 上傳圖片", type="numpy")
            intensity_slider = gr.Slider(
                minimum=0, 
                maximum=100, 
                value=50, 
                step=1, 
                label="卡通化程度"
            )
        with gr.Column():
            output_img = gr.Image(label="2. 卡通化結果")

    intensity_slider.release(
        fn=cartoonize_image_logic, 
        inputs=[input_img, intensity_slider], 
        outputs=output_img
    )
    
    input_img.change(
        fn=cartoonize_image_logic, 
        inputs=[input_img, intensity_slider], 
        outputs=output_img
    )

demo.launch()

Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.
